In [246]:
!pip install tokenizers
!pip install transliterate

In [37]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
import requests
from typing import List, Tuple

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transliterate import translit


# Считывание данных

Сначала разберемся с данными, так как разделителем между `id` и `text_no_spaces` является `,`. И она же встречается в качестве знака препинания встречается в `text_no_spaces`, напишем небольшую функцию, которая заменит первую `,` на `;` и считаем датасет указав `sep=';'`

In [43]:
def my_read_csv(input_file: str)-> pd.DataFrame:
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    with open('clean_dataset.csv', 'w', encoding='utf-8') as outfile:
        for line in lines:
            modified_line = line.replace(',', ';', 1)
            outfile.write(modified_line)

    return pd.read_csv('clean_dataset.csv', sep=';')

In [497]:
df = my_read_csv('dataset_1937770_3.txt')
df.head()

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку


# Создание частотного словаря

Можно было догадаться (посмотреть внимательно на датасет), что в текстах часто встречаются английские слова, причем почти всегда это какие-то бренды. Поэтому логично будет написать функцию, которая принимает на вход наши текста и возвращает набор только брендов + названия этих брендов на кирилице (на всякий случай)



P.S.: Дефолтный перевод в кириллицу работает __фигово__, починим, если останется время.. Вообще кажется, что самый популярный англицизм в данной задаче это __айфон__. Поэтому его добавим руками в словарь (Надеюсь это не бан :) )

In [422]:
def find_and_transliterate_english_words(data):
    eng_words = set()
    translit_eng_words = set()
    for line in data: 
        # Ищем английские слова 
        english_words = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?![a-z])', line)
           
        for word in english_words:
            lower_word = word.lower()
            # Заменяем на кирилицу
            translit_eng = translit(lower_word, 'ru')
            
            eng_words.add(lower_word)
            translit_eng_words.add(translit_eng)
    
    return eng_words | translit_eng_words

In [423]:
find_and_transliterate_english_words(['куплютелевизорPhilipOneMICRI4slimкуплюкроссовкиAdidas,оригинал'])

{'adidas',
 'micri',
 'one',
 'philip',
 'slim',
 'адидас',
 'мицри',
 'оне',
 'пхилип',
 'слим'}

Вроде даже работает

Итак это была подготовка к созданию словаря, в него я загружу словарь русских слов взятый из [гитхаба](https://github.com/danakt/russian-words)

Также добавим в этот словарь все бренды на английском и их кирилицу + АЙФОН

In [499]:
def load_russian_frequency_dict(data):
    url = 'https://raw.githubusercontent.com/danakt/russian-words/master/russian.txt'
    response = requests.get(url)
    words = response.text.splitlines()
    
    brand_words = list(find_and_transliterate_english_words(data))
    brand_words.append('айфон')
    
    numbers = [str(i) for i in range(0, 10000)]
    
    words = set(words + brand_words + numbers)
    return words

# Подготовка токинезатора

Тут даже не знаю, что можно подробно рассказать. 

Создаем токенизатор и загружаем в него корпус из часто употребляемых слов

In [428]:
def load_bpe_tokenizer():
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
    
    # Используем корпус русских слов для обучения
    corpus = ["куплю", "ищу", "cниму", "продам", "бу", "торг", 'айфон']
    
    tokenizer.train_from_iterator(corpus, trainer)
    return tokenizer

Напищем еще функцию, которая разбивает строку на куски, причем в роли сепаратора  цифры и знаки препинания

In [434]:
def tokenize_text(text):
    pattern = r'''
        [A-Z][a-z]+(?:[A-Z][a-z]+)*|  # Слова с CamelCase: HelloWorld, iPhone
        [А-ЯЁ][а-яё]+(?:[А-ЯЁ][а-яё]+)*|  # Русские слова с заглавными: ПриветМир
        [a-z]+|  # английские слова в нижнем регистре
        [а-яё]+|  # русские слова в нижнем регистре
        [A-Z]+|  # XYZ
        [А-ЯЁ]+|  # РУССКИЕ АББРЕВИАТУРЫ
        \d+|  # числа
        [^a-zA-Zа-яА-ЯёЁ\d\s]  # знаки препинания
    '''
    
    tokens = re.findall(pattern, text, re.VERBOSE)
    
    # Дополнительная обработка для случаев, когда слова слиплись без изменения регистра
    refined_tokens = []
    for token in tokens:
        if (re.match(r'^[a-zA-Zа-яА-ЯёЁ]+$', token) and 
            not re.match(r'^([A-ZА-ЯЁ][a-zа-яё]+)+$', token) and
            not token.isupper() and not token.islower()):
            
            subtokens = re.split(r'(?<=[a-zа-яё])(?=[A-ZА-ЯЁ])|(?<=[A-ZА-ЯЁ])(?=[a-zа-яё])', token)
            refined_tokens.extend(subtokens)
        else:
            refined_tokens.append(token)
    
    return refined_tokens

In [435]:
tokenize_text('Сейчас2025годПриветземля,алоалоadsfsdf,dfgdsgSamsundDFsd')

['Сейчас',
 '2025',
 'год',
 'Приветземля',
 ',',
 'алоало',
 'adsfsdf',
 ',',
 'dfgdsg',
 'Samsund',
 'DF',
 'sd']

In [436]:
def enhanced_preprocess_text(text):
    original_text = text
    
    tokens = tokenize_text(text)
    processed_text = ' '.join(tokens)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    
    return processed_text, original_text

Данная функция для восстановления текста в исходный вид, потому что в решении я буду все приводить в нижний регистр

In [489]:
def process_case(text, original_text):
    result = []
    words = text.split()
    original_lower = original_text.lower()
    
    for word in words:
        # Ищем позицию слова в оригинальном тексте
        start_pos = original_lower.find(word.lower())
        if start_pos != -1:
            original_word = original_text[start_pos:start_pos+len(word)]
            if original_word.isupper():
                result.append(word.upper())
            elif original_word[0].isupper() and original_word[1:].islower():
                result.append(word.capitalize())
            else:
                result.append(word)
        else:
            result.append(word)
    
    return ' '.join(result)


# Основная часть. Динамическое программирование 

Что тут происходит по шагам:
1. Инициализация DP
2. Рассматриваем все префиксы строки длины `i` (от 1 до n)
3. Перебираем все возможные места начала последнего слова в префиксе. Для каждой пары `(j, i)` формируем подстроку `word = text_lower[j:i]`
4. Если подстрока целиком есть в словаре нашем построеном словаре `russian_and_brand_dict`, то считаем её "хорошим" словом
5. Если подстрока не найдена в словарe, то пытаемся разобрать подстроку с помощью BPE-токенизатора.
   1. Если токенизатор пометил слово как неизвестно, даём сильный штраф
   2. Если BPE смог разбить подстроку на знакомые подслова, даём небольшое положительное вознаграждение
6. Сохраняем новую лучшую оценку и запомним точку разрыва `j`
7. Обработка односимвольного хвоста (Важно). Потому что как оказалось случай __5B__ может разбить на мелкие слова и оставить один символ в конце предложения, т.е. очевидно неправильное разбиение
8. Вызываем внешнюю функцию `process_case`, которая восстанавливает регистр

In [487]:
def restore_spaces(text: str) -> str:
    original_text = text
    
    text_lower = text.lower()
    
    n = len(text_lower)

    # Инициализация массива динамического программирования
    dp = [-np.inf] * (n + 1)
    dp[0] = 0
    prev = [0] * (n + 1)
    
    for i in range(1, n + 1):
        for j in range(i):
            word = text_lower[j:i]
            
            if word in russian_and_brand_dict:
                score = dp[j] + len(word)**2 
            else:
                # Проверяем через BPE токенизатор
                tokens = bpe_tokenizer.encode(word).tokens
                if len(tokens) == 1 and tokens[0] == '[UNK]':
                    score = -10  
                else:
                    score = dp[j] + 1 
            
            if score > dp[i]:
                dp[i] = score
                prev[i] = j
    
    # Восстанавливаем разбиение
    words = []
    i = n
    while i > 0:
        j = prev[i]
        words.append(text_lower[j:i])
        i = j
    
    words.reverse()
    
    # Обрабатываем случай с одной буквой в конце
    if len(words) > 1 and len(words[-1]) == 1:
        words[-2] += words[-1]
        words.pop()
    
    result = ' '.join(words)
    result = process_case(result, original_text)
        
    return result

In [445]:
def process_dataset(dataset: List[str]) -> List[str]:
    return [restore_spaces(text) for text in dataset]

# Проверка

Тут приведены некоторые ~костыли~ эвристики, если кратко, то чтобы в конце предложения не оставалась одна буква, чтобы обрабатывались окончания, всякие дефисы 

In [493]:
def clean_text_advanced(text):
    patterns = {
        1: (re.compile(r',\s*([а-яё])\s+'), r', \1'),
        2: (re.compile(r'(\w+)\s+(\w)([^\w\s]*)$'), r'\1\2\3'),
        3: (re.compile(r'(\d)\s+(\d)'), r'\1\2'),
        4: (re.compile(r'(\w+)\s+([а-яё])\s*,'), r'\1\2,'),
        5: (re.compile(r'(\w+)\s+(ая|ый|ий|яя|ое|ее|ой|ей|ую|юю|ом|ем|им|ым|ах|ях|ами|ями|ов|ев|ей|ий)\b'), r'\1\2'),
        6: (re.compile(r'-\s+'), '-'),
        7: (re.compile(r"\s*['']\s*"), "'")
    }
    
    patterns_order = [1, 2, 3, 4, 5, 6, 7]
    
    cleaned = text
    
    for pattern_num in patterns_order:
        if pattern_num in patterns:
            pattern, replacement = patterns[pattern_num]
            cleaned = pattern.sub(replacement, cleaned)
    
    # Дополнительная очистка пробелов
    cleaned = re.sub(r'\s*([.,!?;:])\s*', r'\1 ', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()
    
    return cleaned

In [373]:
%%time

text = df['text_no_spaces']

russian_and_brand_dict = load_russian_frequency_dict(text)
bpe_tokenizer = load_bpe_tokenizer()

CPU times: total: 40.8 s
Wall time: 49.7 s


Напишем простенькую функцию проставки идексов пробелов

In [452]:
def get_space_positions(original_text, restored_text):
    restored_no_spaces = restored_text.replace(' ', '')
    
    if restored_no_spaces != original_text:
        # Если тексты не совпадают, пытаемся найти наилучшее соответствие
        # Это может произойти, если есть ошибки в восстановлении
        min_len = min(len(restored_no_spaces), len(original_text))
        restored_no_spaces = restored_no_spaces[:min_len]
        original_text = original_text[:min_len]
    
    space_positions = []
    # i - индекс в restored_text, j - индекс в original_text
    i, j = 0, 0  
    
    while i < len(restored_text) and j < len(original_text):
        if restored_text[i] == ' ':
            space_positions.append(j)
            i += 1
        elif restored_text[i] == original_text[j]:
            i += 1
            j += 1
        else:
            i += 1
    
    return space_positions

In [472]:
%%time

ans_arr = []
restored = process_dataset(text)

for original, pred_result in zip(text, restored):
    result = clean_text_advanced(pred_result)
    ans_arr.append(get_space_positions(original, result))
    # print(f'Original: {original}\nRestored: {result} \n\n')

CPU times: total: 4.95 s
Wall time: 5.05 s


## Что по времени

Получилось достаточно быстро, `bpe_tokenizer` + `inference` = 49.7 s + 5.19 s = 54.89 s (результаты могут отличаться от запуска к запуску, но не сильно)

Вообще специально избегал каких-то больших и страшных моделей, потому что очевидно данная задача должна быстро отрабатывать на инференсе, обучаться желательно тоже, именно поэтому не был загружен словарь английских слов, он тут просто не нужен, достаточно было бренды детектить по "нерусским" буквам + они часто начинаются с большой буквы

In [476]:
df['predicted_positions'] = ans_arr
df.head()

,id,text_no_spaces,predicted_positions
0,0,куплюайфон14про,"[5, 10, 12]"
1,1,ищудомвПодмосковье,"[3, 6, 7, 10, 16]"
2,2,сдаюквартирусмебельюитехникой,"[4, 12, 13, 20, 21]"
3,3,новыйдивандоставканедорого,"[5, 10, 18]"
4,4,отдамдаромкошку,"[5, 10]"


In [495]:
df.to_csv('ans.txt', sep=',', index=False, encoding='utf-8')

Получили `Your Mean F1 = 90.27%`. 

Как можно улучшить:
1. Сделать качественную предобработку, надо было в саааамом начале это делать, я уже не успеваю, думаю процента 3-5 там закопано
2. Может пересмотреть модель и выбрать модель попроще
3. Сделать какую-то вторичное расставление пробелов/удаление пробелов, не просто как я по регуляркам, а что-то УмнОе

---
__Спасибо организаторам__ за такую возможность проявить себя и за такой датасет, под эти песни и пытался разгадать загадку пробелов